# BAC0

BAC0 is a script module based on BACpypes to automate task using Bacnet/IP protocole.

It's still at an early stage but will grow with time.

## How to use

First you must install the 2 modules
- git clone https://github.com/ChristianTremblay/bacpypes-python3.git
- git clone https://github.com/ChristianTremblay/bacnetScript.git

Both are installed using python setup.py install command in the shell
As you can see, it uses BACpypes-python3 which i a port from the official BACpypes Repo which is Python2 (http://bacpypes.sourceforge.net/)

Note : All tests have been done in Windows 7 using Anaconda3 (http://continuum.io/downloads)

## Import module

In [1]:
import BAC0

# You will need to define your local IP address (which is in the same subnet than the devices you're communicating with.
myIPAddr = '192.168.210.63'

bacnet = BAC0.ReadWriteScript(localIPAddr = myIPAddr)

# Wait for the app to start


DEBUG:BAC0.scripts.BasicScript:running
DEBUG:bacpypes.core:run spin=1.0
DEBUG:BAC0.scripts.BasicScript:finally


Starting app...
App started


## Using the application

The application is to be used to test bacnet devices. As it uses BACpypes, there's a lot of different functions that are available (whois, Iam, read, readMultiple, write, etc.) Actually, the basic scripts have been defined to be able to find devices, find points, read to points and write to points.

If you ever want to stop the app and free the socket, you must use :

**bacnet.stopApp()**

If not, you will need to restart the kernel...

### Finding devices on a network

In [2]:
bacnet.whois() # Note that this function is called automatically in ReadWriteScript

# The result takes a few milliseconds to be processed... it can be used in the same cell.

defaultdict(<class 'int'>, {})

In [3]:
# Result can be accessed to print with : 
for each in bacnet.discoveredDevices:
    print(each)


('192.168.210.253', 210253)
('2:5', 5)


This may be not clear enough... what if we would want to know the name of the devices...

In [4]:
# Result can be accessed to print with : 
print('==========================================')
print('Result of the Whois request')
print('==========================================')
for each in bacnet.discoveredDevices:
    deviceName = (bacnet.read('%s device %s objectName' % (each[0], each[1])))
    deviceAddr = each[0]
    print('Found device : %s at address %s' % (deviceName, deviceAddr))

Result of the Whois request
Found device : BASRT-B009157 at address 192.168.210.253
Found device : FX14 0005 at address 2:5


### Read objects from devices

Accessing the list of object of a device can be done using the objectList property


In [2]:
# Let's print the first 10 objects...
bacnet.read('2:5 device 5 objectList')[:10]

[('file', 1),
 ('analogInput', 2),
 ('analogInput', 3),
 ('analogInput', 5),
 ('analogInput', 4),
 ('analogInput', 0),
 ('analogInput', 1),
 ('analogOutput', 0),
 ('analogValue', 28),
 ('analogValue', 29)]

In [3]:
bacnet.readMultiple('2:5 analogInput 1 objectName description presentValue units')

['nvoAI2', 'nvoAI2', 22.100000381469727, 'degreesCelsius']

Doesn't tell you a lot of details... we will have to read values to get them. To accelerate the process, we'll use the special function "discoverPoints" which will return a tuple with a lot of details... and a pandas DataFrame, a good way to show table values.

In [2]:
from BAC0.core.functions.discoverPoints import discoverPoints
r = discoverPoints(bacnet,('2:5'),False)

Found FX14 0005


In [3]:
r[3].head()

,pointType,pointAddress,description,presentValue,units
pointName,,,,,
nvoAI3,analogInput,2,nvoAI3,12.21,degreesCelsius
nvoAI4,analogInput,3,nvoAI4,48.57,percent
nvoAI6,analogInput,5,nvoAI6,365.1,kilopascals
nvoAI5,analogInput,4,nvoAI5,1.85,percent
nvoAI1,analogInput,0,nvoAI1,23.01,degreesCelsius


In [6]:
# r is now a tuple with device name, protocole services supported array, objectList and a DataFrame
# Let's print the head of the DataFrame, you can see that variables have been read

pointsTable = r[3]

pointsTable.head()

,pointType,pointAddress,description,presentValue,units
pointName,,,,,
nvoAI3,analogInput,2,nvoAI3,12.21,degreesCelsius
nvoAI4,analogInput,3,nvoAI4,48.57,percent
nvoAI6,analogInput,5,nvoAI6,365.1,kilopascals
nvoAI5,analogInput,4,nvoAI5,1.85,percent
nvoAI1,analogInput,0,nvoAI1,23.01,degreesCelsius


In [7]:
print(pointsTable.ix['nvoAI3'].pointType)

analogInput


In [8]:
def readPointByName(name):
    device = '2:5'
    val = bacnet.read('%s %s %s presentValue' % (device,pointsTable.ix[name].pointType,str(pointsTable.ix['nvoAI3'].pointAddress)))
    return val

readPointByName('nvoAI4')

12.1899995803833

## Write to points

In [12]:
import time
obj = '2:5 analogValue 41 presentValue'
old_value = bacnet.read(obj)
print('Old Value is : %s' % old_value)
new_value = 120

bacnet.write('%s %s' % (obj,new_value))
time.sleep(1) # Needs a little pause so the write task can be done...
print('New value is %s' % bacnet.read(obj))

DEBUG:BAC0.core.io.Write.WriteProperty:do_write ['2:5', 'analogValue', '41', 'presentValue', '120']


Old Value is : 120.0
    pduSource = <Address 2:5>
    pduExpectingReply = False
    pduNetworkPriority = 0
    apduType = 2
    apduService = 15
    apduInvokeID = 129
    pduData = b''
New value is 120.0


## Stopping the app

In [62]:
bacnet.stopApp()

DEBUG:bacpypes.core:stop


Stopping app
App stopped
